# Ejercicio Covid-19

In [99]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [100]:
#Fechas de hoy(datos España) y ayer (datos inetrnacionales)
from datetime import datetime, timedelta
from datetime import date
date_today = date.today()
date_today_complete = date.today()
date_daybefore = date_today - timedelta(days=1)
date_today = date_today.strftime('%d/%m')
date_daybefore = date_daybefore.strftime('%d/%m')

In [101]:
path_spain = 'pdf_files/Actualizacion_70_COVID-19.pdf'
new_inform = 70
new_before = 69

# Preparación inicial

## Descargar PDFs del Ministerio de Sanidad

In [102]:
#Importar PDFs
import os
import requests
from bs4 import BeautifulSoup
import re

#Función para descargar PDFs
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    print(local_filename)
    return local_filename

#
DAILY_FOLDER = "./pdf_files/"
ES_REPORT_URL = "https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/situacionActual.htm"
PDF_BASE_URL = "https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/"

#Función para revisar los nombres de archivos
def get_file_names(url):
    try:
        req_page = requests.get(url, verify=True)
    except Exception as e:
        raise Exception(f"Could not get web page content: {e}")
    
    re_pdf = re.compile(r"(documentos/.+?\.pdf)")

    pdf_paths = list(set(re_pdf.findall(req_page.text)))
    return pdf_paths

#Reviso los nomnbres de archivos
pdf_paths = get_file_names(ES_REPORT_URL)

pdf_paths = [PDF_BASE_URL+i for i in pdf_paths if "Informacion_inicial_alerta" not in i]


for pdf_path in pdf_paths:
        pdf_name = pdf_path.split("/")[-1]
        pdf_path_get = requests.get(pdf_path)
        with open(
            os.path.join(DAILY_FOLDER, pdf_name),
            'wb'
        ) as f:
            f.write(pdf_path_get.content)
            
# Descargo PDFs a raíz
for i in range(new_before,new_inform):
    path = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/documentos/Actualizacion_{}_COVID-19.pdf'.format(i+1)
    download_file(path)

# Muevo los archivos a otra carpeta
lst_files = [file for file in os.listdir() if 'Act' in file]
if os.path.isdir("./pdf_files/"):
    for f in lst_files:
        os.rename("./{}".format(f), "./pdf_files/{}".format(f))
else:
    os.mkdir("./pdf_data/")
    for f in lst_files:
        os.rename("./{}".format(f), "./pdf_files/{}".format(f))

Actualizacion_70_COVID-19.pdf


## Parsear PDF descargado

In [104]:
# Parsear PDF

from tika import parser
import pandas as pd
from io import StringIO
import re
import tabula

raw = parser.from_file(path_spain)

#Cojo partes del PDF
ind_ini = raw['content'].find('en España:')
ind_fin = raw['content'].find('mundo:')


#Coger fecha del PDF
def get_fecha(string):
    ind_ini = string.find('(COVID-19)')
    ind_fin = string.find('.20')
    print(string[ind_ini:ind_fin])
    return string[ind_ini:ind_fin].split('\n')[-1]+'.2020'

fecha_data_spain = get_fecha(raw['content'])
print(fecha_data_spain)

# Coger tablar primera página
import tabula
from tabula import read_pdf
data_spain_ccaa = tabula.read_pdf(path_spain, stream=True, pages=1)[0]
data_spain_ccaa.columns = ['Comunidades Autónomas','Confirmados', 'IA', 'Hospitalizados', 'UCI', 'Fallecidos', 'Curados', 'Nuevos casos (24h)']
data_spain_ccaa.rename(index={23: "Casos totales en España" + " (act." + date_today + ")" }, inplace=True)
data_spain_ccaa = data_spain_ccaa.iloc[3:22]
data_spain_ccaa = data_spain_ccaa[['Comunidades Autónomas', 'Confirmados', 'Curados', 'Fallecidos', 'Nuevos casos (24h)']]

#Quitar símbolos raros qye nos sobran
data_spain_ccaa['Confirmados'] = data_spain_ccaa['Confirmados'].str.replace('.','')
data_spain_ccaa['Fallecidos'] = data_spain_ccaa['Fallecidos'].str.replace('.','')
data_spain_ccaa['Curados'] = data_spain_ccaa['Curados'].str.replace('.','')
data_spain_ccaa['Nuevos casos (24h)'] = data_spain_ccaa['Nuevos casos (24h)'].str.replace('.','')

# Seleccionar índice Comunidades Autónomas
data_spain_ccaa = data_spain_ccaa.set_index('Comunidades Autónomas')
data_spain_ccaa

(COVID-19).  

09.04
09.04.2020


,Confirmados,Curados,Fallecidos,Nuevos casos (24h)
Comunidades Autónomas,,,,
Andalucía,9261,1193,652,264
Aragón,3685,718,385,136
Asturias,1737,340,110,32
Baleares,1448,696,89,36
Canarias,1834,359,92,72
Cantabria,1619,214,98,47
Castilla La Mancha,12489,1766,1322,701
Castilla y León,10518,3242,1082,460
Cataluña,31043,13063,3148,1396


# Resumen de datos INICIAL

## Resumen datos globales CSV: covid19_data_drive_resume.csv

In [105]:
# Creación de funciones para sacar datos
def get_data_spain(url):    
    data_spain = pd.read_csv(url)
    data_spain = data_spain.groupby("Country/Region", as_index=False).sum()
    data_spain = data_spain.set_index("Country/Region").transpose()
    data_spain = data_spain['Spain']
    data_spain_actual = int(data_spain.iloc[-1])
    data_spain_before = int(data_spain.iloc[-2])
    data_spain_24h = data_spain_actual - data_spain_before
    return data_spain_24h, data_spain_actual
    
# Función para obtener datos del mundo    
def get_data_world(url):    
    data_world = pd.read_csv(url)
    data_world = data_world.groupby("Country/Region", as_index=False).sum()
    data_world = data_world.set_index("Country/Region").transpose()
    data_world_actual = int(data_world.sum(axis=1).iloc[-1])
    data_world_before = int(data_world.sum(axis=1).iloc[-2])
    data_world_24h = data_world_actual - data_world_before
    return data_world_24h, data_world_actual

# Usar los datos
data_confirmed_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
data_deaths_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
data_recovered_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

# Para España
data_confirmed_spain_24h, data_confirmed_spain_actual = get_data_spain(data_confirmed_path)
data_deaths_spain_24h, data_deaths_spain_actual = get_data_spain(data_deaths_path)
data_recovered_spain_24h, data_recovered_spain_actual = get_data_spain(data_recovered_path)

# Para el mundo
data_confirmed_world_24h, data_confirmed_world_actual = get_data_world(data_confirmed_path)
data_deaths_world_24h, data_deaths_world_actual = get_data_world(data_deaths_path)
data_recovered_world_24h, data_recovered_world_actual = get_data_world(data_recovered_path)

#Creo un dataframe por cada dato.
data_spain_newcases = pd.DataFrame({'Confirmados': data_confirmed_spain_24h,
                                    'Curados': data_recovered_spain_24h, 
                                    'Fallecidos': data_deaths_spain_24h}, 
                                   index=['Nuevos casos en España (24h)'])

data_world_newcases = pd.DataFrame({'Confirmados': data_confirmed_world_24h,
                                    'Curados': data_recovered_world_24h, 
                                    'Fallecidos': data_deaths_world_24h}, 
                                   index=['Nuevos casos en el mundo (24h)'])

data_spain_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_spain_actual,
                                    'Curados': data_deaths_spain_actual, 
                                    'Fallecidos': data_recovered_spain_actual}, 
                                   index=['Casos totales en España'])

data_world_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_world_actual,
                                    'Curados': data_recovered_world_actual, 
                                    'Fallecidos': data_deaths_world_actual}, 
                                   index=['Casos totales en el mundo'])

##### Junto los datos anteriores en una tabla
data_resume_final = pd.concat([data_world_resume_actual, data_world_newcases, data_spain_resume_actual, data_spain_newcases])

data_resume_final.to_csv('covid19_data_drive_resume.csv', sep = ';')
data_resume_final

,Confirmados,Curados,Fallecidos
Casos totales en el mundo,1511104,328661,88338
Nuevos casos en el mundo (24h),85008,28607,6473
Casos totales en España,148220,14792,48021
Nuevos casos en España (24h),6278,4813,747


# Datos a nivel mundial

## Preparación general de datos por países

In [106]:
# Datos https://github.com/CSSEGISandData/COVID-19.git
data = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )

#Saco un dataset con solo los países
data_countries = data.groupby("Country/Region", as_index=False).sum()

# Hago transposición de los datos para mover las columnas en filas 
data_trans = data_countries.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_time = data_trans[2:]

# Cogemos solo determinados países
data_time = data_time[['China', 'France', 'Germany', 'Iran','Italy', 'Korea, South', 'Norway','Spain', 'US', 'United Kingdom']]

#Hago listado de Countries
list_countries = data_time.columns
list_countries

# Para coger los países que en día 20 tienen más de 1000 casos: data_countries = data_countries[data_countries["3/17/20"] >= 1000 ]
data_time.tail(2)

#Convierto la columna fecha en datetime
from datetime import date
data_time.index = pd.to_datetime(data_time.index)
data_time.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-04-04,82543.0,90848.0,96092.0,55743.0,124632.0,10156.0,5550.0,126168.0,308850.0,42477.0
2020-04-05,82602.0,93773.0,100123.0,58226.0,128948.0,10237.0,5687.0,131646.0,337072.0,48436.0
2020-04-06,82665.0,98963.0,103374.0,60500.0,132547.0,10284.0,5865.0,136675.0,366667.0,52279.0
2020-04-07,82718.0,110065.0,107663.0,62589.0,135586.0,10331.0,6086.0,141942.0,396223.0,55949.0
2020-04-08,82809.0,113959.0,113296.0,64586.0,139422.0,10384.0,6086.0,148220.0,429052.0,61474.0


## MUNDO: CSV : covid19_data_drive_actual.csv

In [107]:
######### Confirmados

data_confirmed = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )
#Saco un dataset con solo los países
data_confirmed = data_confirmed.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_confirmed = data_confirmed.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_confirmed = data_confirmed[2:]
# Cogemos solo determinados países
data_confirmed = data_confirmed[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_confirmed = data_confirmed[-1:]

######### Muertos

data_deaths = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
#Saco un dataset con solo los países
data_deaths = data_deaths.groupby("Country/Region", as_index=False).sum()
#Se ve que están todos y comprobado que suma
data_deaths[::]
# Hago transposición de los datos para mover las columnas en filas 
data_deaths = data_deaths.set_index('Country/Region').transpose()

# Y me quedo sólo con los datos temporales y número de casos
data_deaths = data_deaths[2:]
data_deaths.tail()
# Cogemos solo determinados países
data_deaths = data_deaths[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_deaths = data_deaths[-1:]

######### Recuperados

data_recovered = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv" )
#Saco un dataset con solo los países
data_recovered = data_recovered.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_recovered = data_recovered.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_recovered = data_recovered[2:]
# Cogemos solo determinados países
data_recovered = data_recovered[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_recovered = data_recovered[-1:]

######### Merge de datos

# Saco la fecha:
from datetime import datetime, timedelta
date_today_intern = date.today()
date_today_intern = date_today_intern.today() - timedelta(days=1) #Quito uno porque esta base de datos siempre muestra datos del día anterior
date_today_intern = date_today_intern.strftime('%d/%m')

#Juntar ahora datos
data_columns = ['Confirmados', 'Curados', 'Fallecidos']

data_together = pd.merge(data_confirmed, data_recovered, how='outer')
data_together = pd.merge(data_together, data_deaths, how='outer')
data_together.index = data_columns
data_together = data_together.transpose()
# data_together = data_together.reindex(columns =data_columns) 
# data_together = data_together.reindex(data_index)

# Exporto CSV
data_together.to_csv('covid19_data_drive_actual.csv', sep = ';')
data_together

,Confirmados,Curados,Fallecidos
Country/Region,,,
China,82809.0,77567.0,3337.0
France,113959.0,21452.0,10887.0
Germany,113296.0,46300.0,2349.0
Italy,139422.0,26491.0,17669.0
Spain,148220.0,48021.0,14792.0
US,429052.0,23559.0,14695.0
United Kingdom,61474.0,345.0,7111.0


## MUNDO: CSV : covid19_data_drive_from100.csv

In [108]:
'''Vamos a hacer un gráfico para ver los casos confirmados (desde caso 100)'''

# Coger solo datos del último mes
data_time_30days = data_time[-200:]
# Si quiero quitar China: data_time_filter = data_time_filter.drop('China', axis=1)

#Exporto los datos en diferentes arhcivos con casa país
for country in list_countries:
    dataX = data_time_30days[country]
    dataX = dataX[dataX > 200]
    dataX.to_csv('data_' + country + ".csv", index=False, header= country)

# Lista dataframes
# dataframes_list = ['dataframe_France', 'dataframe_China', 'dataframe_Germany', 'dataframe_Iran', 'dataframe_Italy', 'dataframe_Korea', 'dataframe_Norway', 'dataframe_Spain', 'dataframe_US', 'dataframe_UnitedKingdom']

# Creamos los CSV
dataframe_France = pd.read_csv('data_France.csv')
dataframe_China = pd.read_csv('data_China.csv')
dataframe_Germany = pd.read_csv('data_Germany.csv')
dataframe_Iran = pd.read_csv('data_Iran.csv')
dataframe_Italy = pd.read_csv('data_Italy.csv')
dataframe_Korea = pd.read_csv('data_Korea, South.csv')
dataframe_Norway = pd.read_csv('data_Norway.csv')
dataframe_Spain = pd.read_csv('data_Spain.csv')
dataframe_US = pd.read_csv('data_US.csv')
dataframe_UnitedKingdom = pd.read_csv('data_United Kingdom.csv')

#Les añadimos NaN al final y reindexamos
new_index = np.arange(1,51)
dataframe_France = dataframe_France.reindex(new_index)
dataframe_China = dataframe_China.reindex(new_index)
dataframe_Germany = dataframe_Germany.reindex(new_index)
dataframe_Iran = dataframe_Iran.reindex(new_index)
dataframe_Italy = dataframe_Italy.reindex(new_index)
dataframe_Korea = dataframe_Korea.reindex(new_index)
dataframe_Norway = dataframe_Norway.reindex(new_index)
dataframe_Spain = dataframe_Spain.reindex(new_index)
dataframe_US = dataframe_US.reindex(new_index)
dataframe_UnitedKingdom = dataframe_UnitedKingdom.reindex(new_index)

#Juntar dataframes y Exportar CSV completo
data_drive_from100cases = dataframe_France.join([dataframe_China, dataframe_Germany, dataframe_Iran, dataframe_Italy, dataframe_Korea, dataframe_Norway, dataframe_Spain, dataframe_US,dataframe_UnitedKingdom])

# Declare a list that is to be converted into a column

'''doubleday = [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25600, 51200, 102400, 204800, 409600, 819200, 
         'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 
         'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 
         'NaN', 'NaN', 'NaN']

# doubletwodays = [100, (100*2)-(100/2), 200, (200*2)-(200/2), 800, (800*2)-(800/2), 1600,(1600*2)-(1600/2), 3200,
                 (3200*2)-(3200/2), 6400, (6400*2)-(6400/2), 12800, (12800*2)-(12800/2), 25600, (25600*2)-(25600/2),
                 51200, (51200*2)-(51200/2), 102400, (102400*2)-(102400/2), 204800, (204800*2)-(204800/2), 409600, (409600*2)-(409600/2), 
        819200, (819200*2)-(819200/2), 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN']

# Using 'Address' as the column name 
# and equating it to the list 
# data_drive_from100cases['Doble cada día'] = doubleday
# data_drive_from100cases['Doble cada dos días'] = doubletwodays
'''

data_drive_from100cases.to_csv('covid19_data_drive_from100.csv', sep = ';')
data_drive_from100cases.tail()

,France,China,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
46,NaN,80823.0,NaN,NaN,NaN,10331.0,NaN,NaN,NaN,NaN
47,NaN,80860.0,NaN,NaN,NaN,10384.0,NaN,NaN,NaN,NaN
48,NaN,80887.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,NaN,80921.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,80932.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## MUNDO: CSV covid19_data_drive_days.csv

In [109]:
# Coger solo datos de últimos 60 días
data_time_Xdays = data_time[-30:]

#Exportar CSV por días progresivos
data_drive_lastX = data_time_Xdays.loc[:,list_countries]
data_drive_lastX.to_csv('covid19_data_drive_days.csv', sep = ';', encoding="utf-8")
data_drive_lastX.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-04-04,82543.0,90848.0,96092.0,55743.0,124632.0,10156.0,5550.0,126168.0,308850.0,42477.0
2020-04-05,82602.0,93773.0,100123.0,58226.0,128948.0,10237.0,5687.0,131646.0,337072.0,48436.0
2020-04-06,82665.0,98963.0,103374.0,60500.0,132547.0,10284.0,5865.0,136675.0,366667.0,52279.0
2020-04-07,82718.0,110065.0,107663.0,62589.0,135586.0,10331.0,6086.0,141942.0,396223.0,55949.0
2020-04-08,82809.0,113959.0,113296.0,64586.0,139422.0,10384.0,6086.0,148220.0,429052.0,61474.0


# Gráficos a nivel España

## ESPAÑA: CSV: covid19_spain_ccaa.csv datos principales

In [110]:
#Exportar CSV
data_spain_ccaa.to_csv('covid19_spain_ccaa.csv', sep = ';', encoding="utf-8")
data_spain_ccaa.head(19)

,Confirmados,Curados,Fallecidos,Nuevos casos (24h)
Comunidades Autónomas,,,,
Andalucía,9261,1193,652,264
Aragón,3685,718,385,136
Asturias,1737,340,110,32
Baleares,1448,696,89,36
Canarias,1834,359,92,72
Cantabria,1619,214,98,47
Castilla La Mancha,12489,1766,1322,701
Castilla y León,10518,3242,1082,460
Cataluña,31043,13063,3148,1396


## ESPAÑA: CSV: covid19_spain_ccaa_lineal.csv

In [111]:
# Casos confirmados por ccaa temporal

# Sacar fecha en formato correcto
fecha_data_spain = get_fecha(raw['content'])
fecha_data_spain = pd.Timestamp(year=int(fecha_data_spain[6:10]), month=int(fecha_data_spain[3:5]), day=int(fecha_data_spain[1:2]), hour=12)
fecha_data_spain = fecha_data_spain.strftime('%Y-%m-%d')
print(fecha_data_spain)

# Cojo la fila de confirmados que me interesa
data_spain_ccaa_last = data_spain_ccaa.copy()
data_spain_ccaa_last.rename(columns = {'Confirmados': fecha_data_spain}, index={'ccaa': 'Comunidades Autónomas'}, inplace = True)
data_spain_ccaa_last = data_spain_ccaa_last[fecha_data_spain]
data_spain_ccaa_last = pd.to_numeric(data_spain_ccaa_last).astype(np.int64)
data_spain_ccaa_last.head()

#HACER MERGE SI NO ESTÁN ESOS DATOS

# Importo los últimos datos recogidos acumulados anteriormente
data_process = pd.read_csv("covid19_spain_ccaa_evolution2.csv", sep = ';', encoding="utf-8")
data_process = data_process.set_index('Comunidades Autónomas')

# Hacer merge de ambos dataframes, con un condicional comprobando si es necesario
if data_process.columns[-1] == data_spain_ccaa_last.name:
    print('La columna ya está: NO hacer merge y no se exporta tabla')
else:
    print('La última columna es diferente y no está: SE HACE MERGE')
    data_process = pd.merge(data_process,data_spain_ccaa_last, left_index=True, right_index=True)
    #Exportar a CSV
    data_process.to_csv('covid19_spain_ccaa_evolution2.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')

data_process

(COVID-19).  

09.04
2020-04-09
La columna ya está: NO hacer merge y no se exporta tabla


,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,...,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09
Comunidades Autónomas,,,,,,,,,,,,,,,,,,,,,
Andalucía,6,12,13,13,13,21,21,21,54,71,...,5818,6392,6972,7374,7869,8301,8581,8767,8997,9261
Aragón,2,2,2,2,2,6,6,6,13,38,...,2272,2491,2700,2889,3078,3232,3347,3449,3549,3685
Asturias,0,1,1,2,5,5,5,5,7,22,...,1236,1322,1384,1433,1522,1605,1646,1679,1705,1737
Baleares,2,2,2,5,6,6,6,6,11,13,...,1069,1131,1204,1257,1271,1293,1320,1369,1412,1448
Canarias,6,7,7,7,8,11,11,11,22,25,...,1262,1380,1444,1490,1564,1622,1649,1725,1762,1834
Cantabria,0,10,10,10,10,10,10,10,12,12,...,1171,1213,1268,1321,1384,1441,1483,1501,1572,1619
Castilla La Mancha,0,3,7,12,13,15,15,15,26,39,...,6424,7047,7682,8523,9324,10031,10602,11077,11788,12489
Castilla y León,2,3,5,11,11,14,14,14,23,56,...,6211,6847,7355,7875,8332,8749,9116,9581,10058,10518
Cataluña,3,15,15,15,24,24,24,24,75,124,...,18773,19991,21804,23460,24734,26032,26824,28323,29647,31043


## ESPAÑA: Análisis por edades : CSV : covid19_spain_ccaa_ages.csv

In [112]:
'''COGER DATOS de DATADISTA dataXXX = pd.read_csv("https://github.com/datadista/datasets/blob/master/COVID%2019/nacional_covid19_rango_edad.csv" )
dataXXX
'''

'COGER DATOS de DATADISTA dataXXX = pd.read_csv("https://github.com/datadista/datasets/blob/master/COVID%2019/nacional_covid19_rango_edad.csv" )\ndataXXX\n'

In [113]:
'''EN PROCESO# data_spain_ccaa_age = tabula.read_pdf('pdf_files/Actualizacion_70_COVID-19.pdf', stream=True, pages=2)[0]
data_spain_ccaa_age_general = data_spain_ccaa_age.iloc[13:24, :]

#data_spain_ccaa_age_general = data_spain_ccaa_age_general['SECRETARIA GENERAL'].str.split(' ',expand=True)
#data_spain_ccaa_age_general = data_spain_ccaa_age_general.iloc [:, [0,1,2,6]]
#data_spain_ccaa_age_general.columns = ['Grupos de edad','Confirmados', 'Hospitalizados', 'Fallecidos']
data90 = data_spain_ccaa_age_general.iloc[[9],[0]]

#data90 = data90[data90['SECRETARIA GENERAL'].str.contains("90 y +")]
# data90 

data90 = data90.replace("90 y +", "90y+")
data90
'''

'EN PROCESO# data_spain_ccaa_age = tabula.read_pdf(\'pdf_files/Actualizacion_70_COVID-19.pdf\', stream=True, pages=2)[0]\ndata_spain_ccaa_age_general = data_spain_ccaa_age.iloc[13:24, :]\n\n#data_spain_ccaa_age_general = data_spain_ccaa_age_general[\'SECRETARIA GENERAL\'].str.split(\' \',expand=True)\n#data_spain_ccaa_age_general = data_spain_ccaa_age_general.iloc [:, [0,1,2,6]]\n#data_spain_ccaa_age_general.columns = [\'Grupos de edad\',\'Confirmados\', \'Hospitalizados\', \'Fallecidos\']\ndata90 = data_spain_ccaa_age_general.iloc[[9],[0]]\n\n#data90 = data90[data90[\'SECRETARIA GENERAL\'].str.contains("90 y +")]\n# data90 \n\ndata90 = data90.replace("90 y +", "90y+")\ndata90\n'

In [118]:

# OJO! CAMBIA FORMATO DEL PDF Y A VECES NO COGE BIEN LA TABLA, DEJAR EN LA 67 DE MOMENTO
#Traigo la segunda tabla
data_spain_ccaa_age = tabula.read_pdf('pdf_files/Actualizacion_67_COVID-19.pdf', stream=True, pages=2)[0]
data_spain_ccaa_age = data_spain_ccaa_age.iloc [:, [0,1,3,7]]

data_spain_ccaa_age.columns = ['Grupos de edad','Confirmados', 'Hospitalizados', 'Fallecidos']
data_spain_ccaa_age['Confirmados'] = data_spain_ccaa_age['Confirmados'].str.replace('.','')
data_spain_ccaa_age['Fallecidos'] = data_spain_ccaa_age['Fallecidos'].str.replace('.','')
data_spain_ccaa_age['Hospitalizados'] = data_spain_ccaa_age['Hospitalizados'].str.replace('.','')
data_spain_ccaa_age.set_index('Grupos de edad', inplace=True)

# Saco por edades general
data_spain_ccaa_age_general = data_spain_ccaa_age.iloc [4:14, :]
data_spain_ccaa_age_general.to_csv('covid19_spain_ccaa_ages.csv', sep = ';', encoding="utf-8")

#Saco por mujeres
data_spain_ccaa_age_mujeres = data_spain_ccaa_age.iloc [22:31, :]
data_spain_ccaa_age_mujeres.to_csv('covid19_spain_ccaa_ages_mujeres.csv', sep = ';', encoding="utf-8")

#Saco por hombres
data_spain_ccaa_age_hombres = data_spain_ccaa_age.iloc [39:49, :]
data_spain_ccaa_age_hombres.to_csv('covid19_spain_ccaa_ages_hombres.csv', sep = ';', encoding="utf-8")


data_spain_ccaa_age_general

,Confirmados,Hospitalizados,Fallecidos
Grupos de edad,,,
0-9,250,7,1
10-19,491,6,1
20-29,4629,53,6
30-39,8761,136,20
40-49,13421,340,50
50-59,16577,643,147
60-69,14933,1080,419
70-79,14300,1094,1341
80-89,11196,115,2121


# Subir archivos a SERVIDOR desde FTP

In [121]:
import csv
from ftplib import FTP
from io import StringIO

In [122]:
# Ayuda: https://www.pythoncircle.com/post/668/uploading-a-file-to-ftp-server-using-python/

passw = pd.read_csv("pass.txt", sep = ',', encoding="utf-8")
host = "akakicreations.com"
username = passw.columns[0]
password = passw.columns[1]

ftp = FTP(host=host)

login_status = ftp.login(user=username, passwd=password)
print(login_status)

print(ftp.dir())
fp = open("covid19_data_drive_days.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_data_drive_days.csv"), fp, 1024)
fp = open("covid19_data_drive_from100.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_data_drive_from100.csv"), fp, 1024)
fp = open("covid19_spain_ccaa.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa.csv"), fp, 1024)
fp = open("covid19_spain_ccaa_evolution2.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa_evolution2.csv"), fp, 1024)
fp = open("covid19_data_drive_actual.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_data_drive_actual.csv"), fp, 1024)
fp = open("covid19_data_drive_resume.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_data_drive_resume.csv"), fp, 1024)
fp = open("covid19_spain_ccaa_ages.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa_ages.csv"), fp, 1024)
fp = open("covid19_spain_ccaa_ages_mujeres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa_ages_mujeres.csv"), fp, 1024)
fp = open("covid19_spain_ccaa_ages_hombres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa_ages_hombres.csv"), fp, 1024)
fp.close()

230-OK. Current restricted directory is /
230 313 Kbytes used (0%) - authorized: 51200 Kb
drwxr-xr-x    2 akakicre   akakicre         4096 Apr  6 08:43 .
drwxr-xr-x    2 akakicre   akakicre         4096 Apr  6 08:43 ..
-rw-------    1 akakicre   akakicre           10 Apr  9 13:05 .ftpquota
-rw-r--r--    1 akakicre   akakicre       260069 Apr  3 12:34 Imagen_github.jpg
-rw-r--r--    1 akakicre   akakicre          265 Apr  9 13:05 covid19_data_drive_actual.csv
-rw-r--r--    1 akakicre   akakicre         2730 Apr  9 13:05 covid19_data_drive_days.csv
-rw-r--r--    1 akakicre   akakicre         3226 Apr  9 13:05 covid19_data_drive_from100.csv
-rw-r--r--    1 akakicre   akakicre          215 Apr  9 13:05 covid19_data_drive_resume.csv
-rw-r--r--    1 akakicre   akakicre          583 Apr  9 13:05 covid19_spain_ccaa.csv
-rw-r--r--    1 akakicre   akakicre          235 Apr  9 13:05 covid19_spain_ccaa_ages.csv
-rw-r--r--    1 akakicre   akakicre          224 Apr  9 13:05 covid19_spain_ccaa_ages_h